<a href="https://colab.research.google.com/github/ggwnsghgg/book_project/blob/main/%EC%B1%85_%EC%B6%94%EC%B2%9C%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymysql
!pip install konlpy

     |████████████████████████████████| 51kB 2.8MB/s 
     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 8.0MB/s 
     |████████████████████████████████| 460kB 46.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [10]:
import pymysql
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import konlpy


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# from google.colab import drive
# drive.mount('/content/drive')



#데이터 가져오기
db = pymysql.connect(host='222.111.24.101', port=3306, user='playdata', password='playdata123!', db='bookdata', charset='utf8')

curs = db.cursor(pymysql.cursors.DictCursor)  

query = '''
select *
    from books_data'''
df_books_data = pd.read_sql(query, db)

In [3]:
# 구글 드라이브 사용시

# path = '/content/drive/MyDrive/data/'
# df_books_data = pd.read_csv(os.path.join(path, 'book_data.csv'), error_bad_lines=False)

b'Skipping line 5: expected 11 fields, saw 12\nSkipping line 69: expected 11 fields, saw 13\nSkipping line 105: expected 11 fields, saw 12\nSkipping line 291: expected 11 fields, saw 12\nSkipping line 899: expected 11 fields, saw 13\nSkipping line 1026: expected 11 fields, saw 12\nSkipping line 1423: expected 11 fields, saw 12\nSkipping line 1806: expected 11 fields, saw 12\nSkipping line 1843: expected 11 fields, saw 13\nSkipping line 2021: expected 11 fields, saw 12\nSkipping line 2022: expected 11 fields, saw 12\nSkipping line 2111: expected 11 fields, saw 12\nSkipping line 2142: expected 11 fields, saw 12\nSkipping line 2249: expected 11 fields, saw 12\nSkipping line 2316: expected 11 fields, saw 12\nSkipping line 2358: expected 11 fields, saw 12\nSkipping line 2410: expected 11 fields, saw 13\nSkipping line 2421: expected 11 fields, saw 12\nSkipping line 2588: expected 11 fields, saw 16\nSkipping line 2753: expected 11 fields, saw 14\nSkipping line 2890: expected 11 fields, saw 12

In [46]:
df_books_data

,id,title,author,pub,date,price,review,rating,story,img,genre
0,1,요리코를 위해,노리즈키 린타로,모모,2020.03,13500,43,9.2,스스로 악마이자 신이 되어 딸을 살해한 남자를 심판한 아버지.비극의 출발이자 마지막...,http://image.yes24.com/goods/89392792/L,추리/미스터리
1,2,그녀는 다 계획이 있다,히가시노 게이고,하빌리스,2021.02,14220,13,9.4,하나야 보석점 살인사건을 둘러싼 과거와 미래의 사건경쾌하고 리듬감 있는 터치의 히가...,http://image.yes24.com/goods/97561319/L,추리/미스터리
2,3,블랙 쇼맨과 이름 없는 마을의 살인,히가시노 게이고,알에이치코리아(RHK),2020.11,16200,54,9.1,"아버지가 살해당했다, 그것도 결혼 소식을 알린 직후에. 행복하다고 느끼는 순간, 비...",http://image.yes24.com/goods/95712763/L,추리/미스터리
3,4,돌이킬 수 없는 약속,야쿠마루 가쿠,북플라자,2017.02,13500,217,8.2,"제51회 에드가와란포상 수상 작가 야쿠마루 가쿠의 미스터리 추리 소설. ""내 딸을 ...",http://image.yes24.com/goods/35552641/L,추리/미스터리
4,5,루팡의 딸,요코제키 다이,북플라자,2019.09,13500,31,8.5,도둑 집안의 대부가 변사체로 발견되었다!도둑 집안의 대부 이와오에게는 하나코라는 손...,http://image.yes24.com/goods/78860364/L,추리/미스터리
...,...,...,...,...,...,...,...,...,...,...,...
79770,79771,하루만 사랑해 2,,파피루스(디앤씨미디어),2003.06,8100,0,0.0,어느 날 서원은 횡단보도에서 교통사고를 당한다. 상대는 인천상고 짱인 노란머 민하루...,http://image.yes24.com/goods/375484/L,로맨스
79771,79772,서울 여자와 대구 남자가 만날 때,정민,영상노트,2003.06,8100,0,0.0,,http://image.yes24.com/goods/376458/L,로맨스
79772,79773,kiss me...,,현무,2003.06,7200,2,5.0,,http://image.yes24.com/goods/377351/L,로맨스
79773,79774,어린 엄마 2,임은희,반디출판사,2003.07,8100,0,0.0,,http://image.yes24.com/goods/377607/L,로맨스


In [55]:
# 장르와 타이틀 값을 이용한 books_df 제작

df = pd.DataFrame(df_books_data['title'], columns=['title'])
df1 = pd.DataFrame(df_books_data['genre'], columns=['genres'])
books_df = pd.concat([df,df1],axis=1)

books_df

,title,genres
0,요리코를 위해,NaN
1,그녀는 다 계획이 있다,NaN
2,블랙 쇼맨과 이름 없는 마을의 살인,NaN
3,돌이킬 수 없는 약속,NaN
4,루팡의 딸,NaN
...,...,...
79770,하루만 사랑해 2,NaN
79771,서울 여자와 대구 남자가 만날 때,NaN
79772,kiss me...,NaN
79773,어린 엄마 2,NaN


,title,genre
0,요리코를 위해,추리/미스터리
1,그녀는 다 계획이 있다,추리/미스터리
2,블랙 쇼맨과 이름 없는 마을의 살인,추리/미스터리
3,돌이킬 수 없는 약속,추리/미스터리
4,루팡의 딸,추리/미스터리
...,...,...
79770,하루만 사랑해 2,로맨스
79771,서울 여자와 대구 남자가 만날 때,로맨스
79772,kiss me...,로맨스
79773,어린 엄마 2,로맨스


In [24]:
df_books_data['genre'].values

array(['추리/미스터리', '추리/미스터리', '추리/미스터리', ..., '로맨스', '로맨스', '로맨스'],
      dtype=object)

In [ ]:
#스토리의 키워드를 뽑기위해 명사와 형용사만 가져오자
story_words = df_books_data['story'].to_numpy()
for i, document in enumerate(story_words):
    okt = konlpy.tag.Okt()
    clean_words = []
    for word in okt.pos(document, stem = True): #어간 뽑자
        if word[1] in ['Noun', 'Adjective']: #명사, 형용사만 가져와
            clean_words.append(word[0])
    #print(clean_words)
    document = ' '.join(clean_words)
    #print(document)
    story_words[i] = document

In [ ]:
#넘파이 배열 데이터프레임 화
df_story_words=pd.DataFrame(story_words,
            columns = ['story_words'])

In [ ]:
df_books = pd.concat([df_books_data, df_story_words], axis = 1)

In [ ]:
df_books.columns

In [ ]:
#날짜, 아이디 날리고
df_books = df_books.drop(['id','date'],axis = 1)

In [ ]:
#혹시모를 중복 제거하자
df_books = df_books.drop_duplicates(['title'])

In [ ]:
df_books.tail()

In [ ]:
df_books = df_books.dropna(axis=0)

In [ ]:
df_books

In [ ]:
from math import log # IDF 계산을 위해

In [ ]:
docs = df_books['genre'] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [ ]:
docs.

In [ ]:
N = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [ ]:
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]        
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

In [ ]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

In [ ]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_